# Individual Exploration Abby: Path 2 - Strategy Execution Analysis

# Path 2 - Strategy Execution Analysis
## INSY6500 - PM Analysis Project

**Purpose:** Explore if any patterns emerge when we compare the planned maintenance activities to historical execution results

**Datasets:**
- `101ki_pm_performance.csv` - 12-month historical performance (April 2024 - March 2025)
- `103ki_pm_forecast.csv` - 12-month PM forecast (April 2026 - March 2027)

---

### Step 1 Import and Set Up

In [2]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

print("Libraries imported")

Libraries imported


### Step 2 Load Performance and Forecast Data

In [8]:
# Define data directory
DATA_DIR = Path('../data')

# Define file paths
PERFORMANCE_FILE = DATA_DIR / '101ki_pm_performance.csv'
FORECAST_FILE = DATA_DIR / '103ki_pm_forecast.csv'

# Load historical performance data
df_performance = pd.read_csv(PERFORMANCE_FILE)
print(f"Performance data shape: {df_performance.shape[0]:,} rows, {df_performance.shape[1]} columns")

#Load scheduled maintenance data
df_forecast = pd.read_csv(FORECAST_FILE, 
                        encoding='cp1252',
                        parse_dates = ['DUE_DATE'],
                        dtype={
                            'INTERVAL': 'category',
                            'JOB_TYPE': 'category',
                            'LABOR_CRAFT': 'category',
                            'PMSCOPETYPE': 'category',
                            'DEPT': 'category',
                            'DEPT_NAME': 'category',
                            'DEPT_TYPE' : 'category',
                            'PLANT' : 'category',
                            'LINE' : 'category',
                            'ZONENAME' : 'category',
                            'PROCESSNAME' : 'category'
                        })
print(f"Forecast data shape: {df_forecast.shape[0]:,} rows, {df_forecast.shape[1]} columns")

Performance data shape: 18,476 rows, 7 columns
Forecast data shape: 131,397 rows, 22 columns


In [86]:
# Checking data upload
print(df_performance.info(), df_performance.isna().sum())
print(df_forecast.info(), df_forecast.isna().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18476 entries, 0 to 18475
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PMNUM                18476 non-null  object 
 1   TIMES_SCHEDULED      18476 non-null  int64  
 2   TIMES_ONTIME         18476 non-null  int64  
 3   TIMES_LATE           18476 non-null  int64  
 4   TIMES_NOT_COMPLETED  18476 non-null  int64  
 5   AVG_PLANNED_HRS      18476 non-null  float64
 6   AVG_ACTUAL_HRS       18476 non-null  float64
dtypes: float64(2), int64(4), object(1)
memory usage: 1010.5+ KB
None PMNUM                  0
TIMES_SCHEDULED        0
TIMES_ONTIME           0
TIMES_LATE             0
TIMES_NOT_COMPLETED    0
AVG_PLANNED_HRS        0
AVG_ACTUAL_HRS         0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131397 entries, 0 to 131396
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------ 

### Step 3  Mergeing Data Sets & Verification

In [98]:
# Data merge done with outer to account for PMNUM difference between data sets to prevent data loss
performance_forecast = pd.merge(df_performance, df_forecast, on='PMNUM', how='outer', indicator=True)

# Checking mergered data set 
performance_forecast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133438 entries, 0 to 133437
Data columns (total 29 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   PMNUM                   133438 non-null  object        
 1   TIMES_SCHEDULED         127526 non-null  float64       
 2   TIMES_ONTIME            127526 non-null  float64       
 3   TIMES_LATE              127526 non-null  float64       
 4   TIMES_NOT_COMPLETED     127526 non-null  float64       
 5   AVG_PLANNED_HRS         127526 non-null  float64       
 6   AVG_ACTUAL_HRS          127526 non-null  float64       
 7   DUE_DATE                131397 non-null  datetime64[ns]
 8   COUNTKEY                131397 non-null  object        
 9   PMDESCRIPTION           131397 non-null  object        
 10  INTERVAL                131397 non-null  category      
 11  FORECASTJP              131324 non-null  object        
 12  JOB_TYPE                131320

In [100]:
# Checking for lost keys
pmnum_lost_from_performance = set(df_performance["PMNUM"]) - set(performance_forecast["PMNUM"])
pmnum_lost_from_forecast   = set(df_forecast["PMNUM"]) - set(performance_forecast["PMNUM"])

print("Keys Lost from performance:", len(pmnum_lost_from_performance))
print("Keys Lost from forecast:", len(pmnum_lost_from_forecast))

# Comparing data set shapes 
print()
print("=" * 80)
print("Merged Data Shape:", f"   Rows: {performance_forecast.shape[0]:,}", f"   Columns: {performance_forecast.shape[1]}")
print("Orginal Performance Shape:", f"   Rows: {df_performance.shape[0]:,}", f"   Columns: {df_performance.shape[1]}")
print("Orginal Forecast Shape:", f"   Rows: {df_forecast.shape[0]:,}", f"   Columns: {df_forecast.shape[1]}")
print("=" * 80)


Keys Lost from performance: 0
Keys Lost from forecast: 0

Merged Data Shape:    Rows: 133,438    Columns: 29
Orginal Performance Shape:    Rows: 18,476    Columns: 7
Orginal Forecast Shape:    Rows: 131,397    Columns: 22


From previous data exploration, we know that:
* 2,041 PMNUMs from last year won't repeat next year (PMNUM only in Performance)
* 1,911 new PMNUMs in Forecast that are not in performance data (PMNUM only in Forecast)
* Performance data set has 28 columns and 133,438 rows
* Forecast data set has 7 columns and 18,476 rows
  
When merging the data sets, we would expect:
* More rows than in either dataset 
* The number of columns to increase by 6 (not including the merge indication column)
* The number of rows to increase by 2,041 (when adding Performance to the Forecast) 

Merge Result Based on Shape: 
* Columns increased by 6 (22 &rarr; 28)
* Rows increased by 2,041 (131,397 &rarr; 133,438)

In [101]:
# Missing data verification 
orig_perf_missing   = df_performance.isna().sum()
orig_forecast_missing = df_forecast.isna().sum()

# Finding PMNUMs missing values from the  performance dataset
performance_missing_mask = performance_forecast['_merge'].isin(['left_only'])
    
pmnum_missing_performance = performance_forecast.loc[performance_missing_mask, 'PMNUM']

# Checking other columns for missing values in the performance  dataset 
performance_cols = df_performance.columns
perf_row_mask = performance_forecast["_merge"].isin(["both", "left_only"])
missing_performance =  performance_forecast.loc[perf_row_mask, performance_cols].isna().sum() 

# Finding PMNUMs' missing values from the forecast dataset
forecast_missing_mask = performance_forecast['_merge'].isin(['right_only'])
pmnum_missing_forecast = performance_forecast.loc[forecast_missing_mask, 'PMNUM']

# Checking other columns for missing values in the forecast dataset 
forecast_cols = df_forecast.columns
forecast_row_mask = performance_forecast["_merge"].isin(["both", "right_only"])
missing_forecast =  performance_forecast.loc[forecast_row_mask, forecast_cols].isna().sum()

print("Missing Data in Merged Dataset:")
print()
print("Number of Maintiance Identifiers Missing Forcast Data:", pmnum_missing_performance.nunique())
print("Number of Maintiance Identifiers Missing Perfromance Data:", pmnum_missing_forecast.nunique())
print()
print("Missing Data in Mergered Dataset form Forecast Data:\n", missing_forecast[missing_forecast > 0])
print("For comparison Orginal Missing: \n", orig_forecast_missing[orig_forecast_missing > 0]) 
print()
print("Missing Data in Mergered Dataset form Performance Data:\n", missing_performance)
print("For comparison Orginal Missing:\n",orig_perf_missing)




Missing Data in Merged Dataset:

Number of Maintiance Identifiers Missing Forcast Data: 2041
Number of Maintiance Identifiers Missing Perfromance Data: 1911

Missing Data in Mergered Dataset form Forecast Data:
 FORECASTJP                    73
JOB_TYPE                      77
LABOR_CRAFT                26914
PLANNED_LABOR_HRS            829
TOTAL_MATERIAL_COST       129122
TOTAL_TASK_DESC_LENGTH       104
DEPT_NAME                  14982
LINE                        2791
PROCESSNAME                14740
dtype: int64
For comparison Orginal Missing: 
 FORECASTJP                    73
JOB_TYPE                      77
LABOR_CRAFT                26914
PLANNED_LABOR_HRS            829
TOTAL_MATERIAL_COST       129122
TOTAL_TASK_DESC_LENGTH       104
DEPT_NAME                  14982
LINE                        2791
PROCESSNAME                14740
dtype: int64

Missing Data in Mergered Dataset form Performance Data:
 PMNUM                  0
TIMES_SCHEDULED        0
TIMES_ONTIME           0
T

#### Takeaways 
* Missing data in the merged data set matches that of the original data sets when looking at the merged data set. 
* No data was lost during the merge. 
* Merge was successful.

### Step 4 Data Quality Analysis & Data Cleaning 

In [107]:
# Data Type verification 
performance_forecast.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133438 entries, 0 to 133437
Data columns (total 29 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   PMNUM                   133438 non-null  object        
 1   TIMES_SCHEDULED         127526 non-null  float64       
 2   TIMES_ONTIME            127526 non-null  float64       
 3   TIMES_LATE              127526 non-null  float64       
 4   TIMES_NOT_COMPLETED     127526 non-null  float64       
 5   AVG_PLANNED_HRS         127526 non-null  float64       
 6   AVG_ACTUAL_HRS          127526 non-null  float64       
 7   DUE_DATE                131397 non-null  datetime64[ns]
 8   COUNTKEY                131397 non-null  object        
 9   PMDESCRIPTION           131397 non-null  object        
 10  INTERVAL                131397 non-null  category      
 11  FORECASTJP              131324 non-null  object        
 12  JOB_TYPE                131320

#### Takeaway
* Data types match the expexed types 

In [110]:
# Missing Data Analysis
missing_mask = performance_forecast.isna()
missing = missing_mask.sum().to_frame(name="count")
missing['precentage'] = (missing['count']/performance_forecast.shape[0])

missing.style.format({'precentage': "{:.1%}"})

,count,precentage
PMNUM,0,0.0%
TIMES_SCHEDULED,5912,4.4%
TIMES_ONTIME,5912,4.4%
TIMES_LATE,5912,4.4%
TIMES_NOT_COMPLETED,5912,4.4%
AVG_PLANNED_HRS,5912,4.4%
AVG_ACTUAL_HRS,5912,4.4%
DUE_DATE,2041,1.5%
COUNTKEY,2041,1.5%
PMDESCRIPTION,2041,1.5%


### Takeaways
* The missing data percentage is relatively low, between 1.5% and 4.5% for most of the data
* Process Name and  Department name  have about 12% missing data
    * Both are strings that provide names, less important for analysis
    * Still usable for analysis
    * Department name has additional data with lower missing percentages that can be used for analysis and to fill in missing data
* Labor Craft has about 25% missing data
    * Categorizes the required labor skill/craft to perform maintenance
    * Limits categorical analysis
* Total Material Cost has 98% of the data missing
    * Unusable 

In [115]:
# Duplication detection 
performance_forecast_dups = performance_forecast.nunique().to_frame(name = 'unique_vals')
performance_forecast_dups['duplicates'] = performance_forecast.shape[0] - performance_forecast_dups['unique_vals']
performance_forecast_dups['precentage'] = performance_forecast_dups['duplicates'] / performance_forecast.shape[0]

performance_forecast_dups.style.format({'precentage': "{:.1%}"})

,unique_vals,duplicates,precentage
PMNUM,20387,113051,84.7%
TIMES_SCHEDULED,50,133388,100.0%
TIMES_ONTIME,67,133371,99.9%
TIMES_LATE,26,133412,100.0%
TIMES_NOT_COMPLETED,10,133428,100.0%
AVG_PLANNED_HRS,389,133049,99.7%
AVG_ACTUAL_HRS,1381,132057,99.0%
DUE_DATE,364,133074,99.7%
COUNTKEY,89210,44228,33.1%
PMDESCRIPTION,17380,116058,87.0%


#### Takeaways 
* Due to the nature of the forecast data, duplication is expected